_Anmerkung: Ggf. müssen sie nach dem Öffnen des Notebooks noch einmal auf 'Kernel > Restart & Run All' clicken_

<h1>Abfragen mit SPARQL</h1>

In diesem Demonstrator werden wir zeigen, wie SPARQL Abfragen genutzt werden können, 
um aus Open Data Beständen weiteres Wissen zu ziehen.

![SPARQL Logo](./media/SPARQL.png)

Zuerst: Was ist SPARQL?
Bei SPARQL handelt es sich um die **S**PARQL **P**rotocol **A**nd **R**DF **Q**uery **L**anguage.
Es ist also sowohl ein Protokoll als auch eine Abfragesprache für das Resource Description Framework. 
Während SQL die Abfragesprache für relationale Datenbanken ist, ist SPARQL die geläufige Abfragesprache für RDF-Daten, welche nicht als Tabellen sondern als Graph mit gelabelten Kanten zu verstehen sind.

Eine SPARQL Abfrage ähnelt in ihren Augen möglicherweise einer SQL-Abfrage, jedoch sind die Gemeinsamkeiten nur oberflächlich.
Abfragen besitzen einen SELECT-Part, der die resultierenden Variablen (von der Struktur den Spalten in SQL entsprechend) wiedergibt und einem WHERE-Part, der definiert, welche Eigenschaften die resultierenden Filme alle gemeinsam besitzen.

In [2]:
from data.sparql import query, table

table(query("""
SELECT ?movie
WHERE
{
 ?movie wdt:P31 wd:Q11424
}
LIMIT 10
"""))

| movie |
| --- |
| http://www.wikidata.org/entity/Q372 |
| http://www.wikidata.org/entity/Q593 |
| http://www.wikidata.org/entity/Q595 |
| http://www.wikidata.org/entity/Q1365 |
| http://www.wikidata.org/entity/Q2201 |
| http://www.wikidata.org/entity/Q2345 |
| http://www.wikidata.org/entity/Q2875 |
| http://www.wikidata.org/entity/Q3092 |
| http://www.wikidata.org/entity/Q3187 |
| http://www.wikidata.org/entity/Q3208 |


'showing 10 results'

Diese Query liefert die ersten 10 Filme, welche in Wikidata gelistet sind.
Zwar können sie auf die Einträge klicken, um zu sehen, um was es sich handelt, jedoch ist dieses Format noch nicht sonderlich leserlich.

Um statt der Objekte selbst ihre Label zu erhalten, erweitern wir die vorherige Query. Diesmal soll nicht der Film selbst, sondern sein Titel angezeigt werden.

In [3]:
from data.sparql import query, table

table(query(
"""
SELECT ?title
WHERE
{
 ?movie wdt:P31 wd:Q11424 .
 ?movie wdt:P1448 ?title       
} 
LIMIT 10
"""
))    

| title |
| --- |
| ...men Olsenbanden var ikke død |
| Le guerriere dal seno nudo |
| La vendetta dei gladiatori |
| Relatives |
| Zinebi Festival Internacional de Cine Documental y Cortometraje de Bilbao |
| Börn |
| Syurpriz |
| Petualang Tak Kenal Menyerah |
| طلق صناعي |
| Brexit (2019) |


'showing 10 results'

Wie sie sehen, muss nicht jede Variable auch ausgegeben werden.
Hier sieht man nur den Titel, aber nicht das Film-Objekt.

Zudem ist dies die erste Query, bei der mehrere Kriterien (nämlich Objekt ?movie ist vom Typ Film und Object ?movie hat Titel ?title.
Jedes dieser logischen Tripletts wird durch einen Punkt vom nächsten getrennt.
Um etwas Schreibarbeit zu sparen, kann auch 

```
SELECT ?title
WHERE
{
 ?movie wdt:P31 wd:Q11424;
        wdt:P1448 ?title       
} 
LIMIT 10
```

schreiben. Hier wird durch das Semikolon ";" das Subjekt aus dem letzten Triplet für das Nächste wiederverwendet.

Schreiben sie ebenfalls eine Abfrage, welche die IMDB ID dreier Filme ausgibt.
Das Prädikat für 'hat IMDB ID' lautet 'wdt:P345', bei der IMDB ID handelt es sich um eine ID der 'Internet Movie Database'.

In [4]:
# Tragen sie hier ihren SPARQL-Code ein und klicken sie auf RUN, um ihn auszutesten.
# Denken sie daran, ggf. über 'from data.sparql import query, table' die notwendigen Funktionen zu importieren,
# sollte es sie hier zu einem Fehler kommen.








Die durch SPARQL verfügbar gemachten Daten können nun verwendet werden, 
um beispielsweise zusätzliche Informationen zu denen einer Datenbank hinzuzufügen.
In diesem Notebook handelt es sich hierbei um eine MongoDB, welche eine vielzahl an Filmen enthält.

In [6]:
import data.movies as movies

movies.get_by_title('The Lion King')

{'index': 359,
 'adult': 'False',
 'belongs_to_collection': "{'id': 94032, 'name': 'The Lion King Collection', 'poster_path': '/oG1heUs503GV6xBnFhgyYTGWm05.jpg', 'backdrop_path': '/A9IEaj9cPAwEgFnTmteB70oxgJY.jpg'}",
 'budget': '45000000',
 'genres': "[{'id': 10751, 'name': 'Family'}, {'id': 16, 'name': 'Animation'}, {'id': 18, 'name': 'Drama'}]",
 'homepage': 'http://movies.disney.com/the-lion-king',
 'id': '8587',
 'imdb_id': 'tt0110357',
 'original_language': 'en',
 'original_title': 'The Lion King',
 'overview': "A young lion cub named Simba can't wait to be king. But his uncle craves the title for himself and will stop at nothing to get it.",
 'popularity': '21.605761',
 'poster_path': '/bKPtXn9n4M4s8vvZrbw40mYsefB.jpg',
 'production_companies': "[{'name': 'Walt Disney Pictures', 'id': 2}, {'name': 'Walt Disney Feature Animation', 'id': 10217}]",
 'production_countries': "[{'iso_3166_1': 'US', 'name': 'United States of America'}]",
 'release_date': '1994-06-23',
 'revenue': 788241

Über den Namen des Films oder andere Attribute wie die IMDB-ID können weitere Informationen gesucht werden:

In [7]:
from data.sparql import query, data, table, entity_to_json

result = query(
"""
SELECT *
WHERE
{
 ?movie wdt:P31 wd:Q11424 ;
        rdfs:label ?title ;
        wdt:P345 ?imdb_id .
  FILTER CONTAINS(?title, "The Lion King")
  FILTER LANGMATCHES(lang(?title),'en')
} 
LIMIT 1
"""
)

# a bit hacky, but it retrieves the id from the wikidata URI, which has the form https://www.wikidata.org/wiki/{id}
entity_id = data(result)[0]['movie'].split("/").pop()

entity_to_json(entity_id)

{'entities': {'Q27044293': {'pageid': 28914694,
   'ns': 0,
   'title': 'Q27044293',
   'lastrevid': 1429021083,
   'modified': '2021-05-26T09:42:00Z',
   'type': 'item',
   'id': 'Q27044293',
   'labels': {'en': {'language': 'en', 'value': 'The Lion King'},
    'de': {'language': 'de', 'value': 'Der König der Löwen'},
    'ru': {'language': 'ru', 'value': 'Король Лев'},
    'fi': {'language': 'fi', 'value': 'Leijonakuningas'},
    'he': {'language': 'he', 'value': 'מלך האריות'},
    'zh': {'language': 'zh', 'value': '獅子王'},
    'es': {'language': 'es', 'value': 'El rey león'},
    'ar': {'language': 'ar', 'value': 'الأسد الملك'},
    'ko': {'language': 'ko', 'value': '라이온 킹'},
    'uk': {'language': 'uk', 'value': 'Король Лев'},
    'hu': {'language': 'hu', 'value': 'Az oroszlánkirály'},
    'pt': {'language': 'pt', 'value': 'O Rei Leão'},
    'fr': {'language': 'fr', 'value': 'Le Roi lion'},
    'fa': {'language': 'fa', 'value': 'شیرشاه'},
    'pl': {'language': 'pl', 'value': 'Król 

Hier erkennt man, dass wir eine große Menge an Daten zum Film erhalten. 
Wir können hier den Film-Titel in vielen Sprachen angeben, herausfinden, welcher Direktor für den Schnitt des Filmes verantwortlich ist, welche unterschiedlichen Jugendschutz-Label ein Film hat, etc.

Einige der Eigenschaften sind jedoch über ihre Wikidata-ID aufgelistet. Diese muss aufgelöst werden, um menschenverständliche Daten zu erhalten, also z.B. das es sich bei 'P345' um eine 'IMDB ID' handelt. 